In [13]:
import pandas as pd
import numpy as np
import re
import pandas as pd
import numpy as np
from nltk.metrics.distance import edit_distance

In [4]:
path = '/export/usuarios_ml4ds/cggamella/NP-Company-Process/data/metadata/outsiders.parquet'
df = pd.read_parquet(path)

In [6]:
def unify_colname(col):
    return ".".join([el for el in col if el])

In [7]:
df.columns = [unify_colname(col) for col in df.columns]

In [8]:
df

id  \
zip                                       file name                                          entry                                                      
PlataformasAgregadasSinMenores_2018.zip   PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                                                             452    https://contrataciondelestado.es/sindicacion/P...   
                                                                                             451    https://contrataciondelestado.es/sindicacion/P...   
                                                                                             448    https://contrataciondelestado.es/sindicacion/P...   
                                                                                             447    https://contrataciondelestado.es/sindicacion/P...   
...                                                                                                                                               ...   
PlataformasAgregadasSinMenores_202301.zip PlataformasAgregadasSinMenores.atom                4      https://contrataciondelestado.es/sindicacion/P...   
                                                                                             3      https://contrataciondelestado.es/sindicacion/P...   
                                                                                             2      https://contrataciondelestado.es/sindicacion/P...   
                                                                                             1      https://contrataciondelestado.es/sindicacion/P...   
                                                                                             0      https://contrataciondelestado.es/sindicacion/P...   

                                                                                                                                              summary  \
zip                                       file name                                          entry                                                      
PlataformasAgregadasSinMenores_2018.zip   PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                                                             452    Expediente: 1282/17, Entidad: Diputación Provi...   
                                                                                             451    Expediente: 1281/17, Entidad: Diputación Provi...   
                                                                                             448    Id licitación: 2017017;  Órgano de Contratació...   
                                                                                             447    Id licitación: B2017002;  Órgano de Contrataci...   
...                                                                                                                                               ...   
PlataformasAgregadasSinMenores_202301.zip PlataformasAgregadasSinMenores.atom                4      Id licitación: 8328_01/2023; Órgano de Contrat...   
                                                                                             3      Id licitación: 1004_0005-CP01-2022-000101; Órg...   
                                                                                             2      Id licitación: 8129_PL23/0001; Órgano de Contr...   
                                                                                             1      Id licitación: 3006_1/2023; Órgano de Contrata...   
                                                                                             0      Id licitación: CONTR 2022 0001072929; Órgano d...   

                                                                                                                                                title  \
zip           

In [32]:
def analizar_similitud(df):
    # Aseguramos que los textos son cadenas
    df['title'] = df['title'].astype(str)
    df['name'] = df['ContractFolderStatus.ProcurementProject.Name'].astype(str)

    # Calculamos la distancia de Levenshtein
    df['similarity'] = df.apply(lambda x: 1 - (edit_distance(x['title'], x['name']) / max(len(x['title']), len(x['name']))), axis=1)

    # Calculamos la longitud media de cada texto
    longitud_media_title = df['title'].apply(len).mean()
    longitud_media_objeto_contrato = df['name'].apply(len).mean()

    return {
        'Longitud Media Title': longitud_media_title,
        'Longitud Media ObjetoContrato': longitud_media_objeto_contrato,
    }

In [55]:
resultados = analizar_similitud(df.iloc[0:50000])
print(resultados)

/tmp/ipykernel_2968393/763629721.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].astype(str)
/tmp/ipykernel_2968393/763629721.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df['ContractFolderStatus.ProcurementProject.Name'].astype(str)


{'Longitud Media Title': 164.26688, 'Longitud Media ObjetoContrato': 166.45802}


/tmp/ipykernel_2968393/763629721.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarity'] = df.apply(lambda x: 1 - (edit_distance(x['title'], x['name']) / max(len(x['title']), len(x['name']))), axis=1)


In [56]:
if 'title' in df.columns and 'ContractFolderStatus.ProcurementProject.Name' in df.columns:
    # Crear un nuevo DataFrame con solo las columnas deseadas
    df_reducido = df[['title', 'ContractFolderStatus.ProcurementProject.Name']].copy()

    # Muestreo aleatorio de 4000 filas
    df_muestra = df_reducido.sample(n=4000, random_state=1)  # random_state para reproducibilidad
else:
    print("Error: El DataFrame no contiene las columnas necesarias.")

In [57]:
df_muestra.reset_index(drop=True, inplace=True)

In [58]:
df_muestra

,title,ContractFolderStatus.ProcurementProject.Name
0,"Constitueix l'objecte d'aquesta licitació la contractació del subministrament, transport i descàrrega de compostadors per a la campanya del compostatge casolà que es portarà a terme a diferents municipis del Gironès.\n\n","Constitueix l'objecte d'aquesta licitació la contractació del subministrament, transport i descàrrega de compostadors per a la campanya del compostatge casolà que es portarà a terme a diferents municipis del Gironès."
1,"Subministrament del material descrit al ""Plec de prescripcions tècniques per l'adjudicació del contracte de subministrament de roba de treball pel personal de l'Ajuntament de Cambrils i els seus organismes autònoms"".","Subministrament del material descrit al ""Plec de prescripcions tècniques per l'adjudicació del contracte de subministrament de roba de treball pel personal de l'Ajuntament de Cambrils i els seus organismes autònoms""."
2,"Suministro de trípticos, vinilos y pegatinas Barik para la red comercial Barik y Oficinas integrales del Transporte Público de Bizkaia","Suministro de trípticos, vinilos y pegatinas Barik para la red comercial Barik y Oficinas integrales del Transporte Público de Bizkaia"
3,L'objecte del contracte és la prestació del servei de depuració i control d'aigua de les instal·lacions de l'OE.,L'objecte del contracte és la prestació del servei de depuració i control d'aigua de les instal·lacions de l'OE.
4,"Limpieza en diferentes inmuebles adscritos a la Dirección General de Educación Secundaria, Formación Profesional y Régimen Especial de la Comunidad de Madrid, dividido en dos lotes","Limpieza en diferentes inmuebles adscritos a la Dirección General de Educación Secundaria, Formación Profesional y Régimen Especial de la Comunidad de Madrid, dividido en dos lotes"
...,...,...
3995,"Execució de les obres RAM 2021, als Serveis Territorials al Baix Llobregat (VI): Institut Martí i Dot (Sant Feliu de Llobregat) i Escola Sant Jordi (Sant Vicenç dels Horts). Clau: XMA-21525","Execució de les obres RAM 2021, als Serveis Territorials al Baix Llobregat (VI): Institut Martí i Dot (Sant Feliu de Llobregat) i Escola Sant Jordi (Sant Vicenç dels Horts). Clau: XMA-21525"
3996,"suministro, instalación y puesta en marcha de una unidad de sistema de alimentación ininterrumpida (sai) para el cpd de la sede del estadio deportivo la cartuja, sevilla, de la empresa pública para la gestión del turismo y del deporte de andalucía, s.a.","suministro, instalación y puesta en marcha de una unidad de sistema de alimentación ininterrumpida (sai) para el cpd de la sede del estadio deportivo la cartuja, sevilla, de la empresa pública para la gestión del turismo y del deporte de andalucía, s.a."
3997,"L'objecte del present contracte és evitar l'accés al recinte de qualsevol persona aliena, tret d'aquelles que ho facin per raons de manteniment i amb els corresponents mitjans de seguretat. S'inclouen els treballs necessaris per realitzar el tancament perimetral del recinte del Castell de Can Feu amb la singularitat de cada zona així com el tapiat de diverses obertures de les façanes i porxos amb paret de maó ceràmic.","L'objecte del present contracte és evitar l'accés al recinte de qualsevol persona aliena, tret d'aquelles que ho facin per raons de manteniment i amb els corresponents mitjans de seguretat. S'inclouen els treballs necessaris per realitzar el tancament perimetral del recinte del Castell de Can Feu amb la singularitat de cada zona així com el tapiat de diverses obertures de les façanes i porxos amb paret de maó ceràmic."
3998,"Subministrament de 950 metres de cable tipus CERVIFLEX DN-F 0.6/1KV 1X150mm2 i d'una bomba submergible per pou de 400m3/h i 150 metres d'alçada, ambdós per instal·lar al Pou Renau","Subministrament de 950 metres de cable tipus CERVIFLEX DN-F 0.6/1KV 1X150mm2 i d'una bomba submergible per pou de 400m3/h i 150 metres d'alçada, ambdós per instal·lar al Pou Renau"


In [59]:
def normalizar_texto(texto):
    return texto.str.lower().str.strip().str.replace(r'\W', '', regex=True)

def comparar_columnas(df, col1, col2):
    try:
        if col1 in df.columns and col2 in df.columns:
            # Normalizar las columnas
            norm_col1 = normalizar_texto(df[col1])
            norm_col2 = normalizar_texto(df[col2])

            # Utilizar numpy para comparar eficientemente
            are_identical = np.array_equal(norm_col1, norm_col2)

            if are_identical:
                print("Las columnas son idénticas después de la normalización.")
            else:
                print("Las columnas NO son idénticas después de la normalización.")
                
                # Filtrar las filas que no son idénticas
                df_no_identical = df[norm_col1 != norm_col2]
                print("Filas que no son idénticas después de la normalización:")
                print(df_no_identical)
        else:
            print("Error: El DataFrame no contiene las columnas necesarias.")
    except Exception as e:
        print(f"Se produjo un error al comparar las columnas: {e}")

In [60]:
comparar_columnas(df_muestra, 'title', 'ContractFolderStatus.ProcurementProject.Name')

Las columnas NO son idénticas después de la normalización.
Filas que no son idénticas después de la normalización:
                                                                                                                                                                                                   title  \
5     Servicio de diseño gráfico y creatividad de los soportes para campañas de comunicación, actos públicos y demás actividades de la Secretaría General de Derechos Humanos, Convivencia y Cooperación   
15                                                                                                                                                                         PA 1_21 Mamografias Digitales   
16                                                                                              Servicio de mejora en los sistemas de gestión, seguimiento y control de las prestaciones de la RGI Y PCV   
28                                                                   

In [61]:
df_no_identical.iloc[20:40]

,title,ContractFolderStatus.ProcurementProject.Name
83,Subministrament de 62 columnes elevadores sense fil,Subministrament de 62 columnes elevadores sense fil
85,"Prevención de incendios en Las Matas. Concejos Valdés, Coaña y Navia","Mantenimiento de pistas y fajas auxiliares en Cadavedo y Pedredos, mantenimiento de pistas y mantenimiento de cortafuegos en Cordal de Coaña, mantenimiento de pista en Anleo y cortafuegos en Castañedo."
92,CONTRATACIÓN DEL SERVICIO DE LA REDACCIÓN DEL PROYECTO DE EJECUCIÓN DE REFORMA DE LA ZONA DE PISCINAS DE MAJORI KIROLDEGIA DE ORDIZIA,Redacción de proyecto de ejecución de reforma de la zona de piscinas del polideportivo Majori
93,2020-EXTERNALIZACION-NOC,"El objeto del presente procedimiento de licitación es la contratación de un servicio de mantenimiento, monitorización y gestión de la red, tanto óptica como IP, de REDIMadrid, también se incluyen 3 consultas técnicas sobre equipamiento instalado en REDIMadrid."
95,Medicamentos exclusivos Janssen Cilag para el Hospital Universitario de Getafe,Suministro de medicamentos exclusivos de Janssen Cilag para el Hospital Universitario de Getafe
101,Contractació dels serveis de transport internacional de components sanguinis,Contractació dels serveis de transport internacional de components sanguinis
108,Adecuación de las instalaciones existentes del tranvía de Vitoria-Gasteiz a unidades mayores - Señalización Viaria,Adecuación de las instalaciones existentes del tranvía de Vitoria-Gasteiz a unidades mayores &#8211; Señalización Viaria
111,L'objecte del contracte és el servei de transport escolar mitjançant vehicle de tipus turisme amb conductor amb criteris de sostenibilitat de la ruta PTEC03.\n\n,L'objecte del contracte és el servei de transport escolar mitjançant vehicle de tipus turisme amb conductor amb criteris de sostenibilitat de la ruta PTEC03.
119,Obras de reforma y sustitución del sistema de producción de frio para climatización del edificio de la sede central del SUMMA 112,Obras de reforma y sustitución del sistema de climatización del edificio de la sede central del SUMMA 112
129,"En funció de les necessitats de cada camí, en el document tècnic aprovat, es preveuen actuacions per a formació de noves pendents per treure les aigües, aportació de materials, neteja i formacio de cunetes i formació d'una escullera de contenció per estabilitzar el terreny en un dels camins. S'actúa en els següents camins: Cami de Sant Genís al Pont de Cal Mateu, Camí de Sant Genís a Rubio, Cami al Pla de Sant Salvador, Cami de Cal Tetes i Camí de Les Muntades.","En funció de les necessitats de cada camí, en el document tècnic aprovat, es preveuen actuacions per a formació de noves pendents per treure les aigües, aportació de materials, neteja i formacio de cunetes i formació d'una escullera de contenció per estabilitzar el terreny en un dels camins. S'actúa en els següents camins: Cami de Sant Genís al Pont de Cal Mateu, Camí de Sant Genís a Rubio, Cami al Pla de Sant Salvador, Cami de Cal Tetes i Camí de Les Muntades."


In [26]:
pd.set_option('display.max_colwidth', None)